In [ ]:
readFiles = False
%run ./init.ipynb

In [ ]:
def renameFileIfExist(fileName):
    if os.path.exists(fileName):
        os.rename(fileName,'old_' + fileName)

In [ ]:
listing = {}

In [ ]:
%%time
startRange = 1
stopRange = 100001

rowAttributes = {}
refStartx = startRange

for x in range(startRange, stopRange):
    scraper = cloudscraper.create_scraper(
        delay=.1,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x)).text
    soup = BeautifulSoup(html, 'lxml')
    rows = soup.find_all('div', 'label')

    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
            
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = ""
        
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    #output_list = [(rowAttributes[key]) for key in (manualColumns) if key in rowAttributes]
    output_list = [(rowAttributes[key]) for key in (manualColumns)]
    listing[str(x)] = output_list
    clear_output(wait=True)
    #print(output_list)
    print(x)
    if(x % 5000 == 0):
        if refStartx == 1:
            refStartx = 0
        data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
        fileName = 'Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv'
        renameFileIfExist(fileName)
        data_dictionary.to_csv(fileName)
        refStartx = x
        listing.clear()

In [ ]:
fileName = 'Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv'
renameFileIfExist(fileName)
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv(fileName)

In [ ]:
for count in range(0, len(manualColumns)):
    print(count)
    print(data_dictionary[manualColumns[count]].value_counts())
    data_dictionary[manualColumns[count]].value_counts(normalize=True)
    print('---------------------------------')

In [ ]:
print('done')